In [1]:
import os,sys
from dotenv import find_dotenv,load_dotenv
load_dotenv(find_dotenv())
src_path = os.environ.get('PROJECT_SRC')
os.chdir(src_path)

In [2]:
import pypsa
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
plt.style.use('ggplot')
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature
import data_reader_writer

# Setting up simulation period

We need to set the required inputs to simulate the LOPF such as the start data, end date and simualtion snaphsots

In [3]:
My_network=pypsa.Network()
My_network.import_from_csv_folder('LOPF_data')

INFO:pypsa.components:Applying weightings to all columns of `snapshot_weightings`
INFO:pypsa.io:Imported network LOPF_data has buses, generators, lines, loads, storage_units


In [4]:
My_network.loads_t.p_set

Load,Beauly,Peterhead,Errochty,Denny/Bonnybridge,Neilston,Strathaven,Torness,Eccles,Harker,Stella West,...,Bramford,Pelham,Sundon/East Claydon,Melksham,Bramley,London,Kemsley,Sellindge,Lovedean,S.W.Penisula
snapshot,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,200.584386,263.372404,76.372672,655.496339,812.336302,217.503617,199.788441,62.109171,274.734376,1495.505149,...,724.917053,415.380352,1495.274816,2267.290753,709.303040,4745.983241,774.605384,398.207590,1043.280962,1382.283119
2022-01-01 01:00:00,179.444630,235.615365,68.323692,586.413030,726.723498,194.580729,178.732569,55.563434,245.779890,1337.892606,...,648.517437,371.603069,1337.686549,2028.339079,634.548998,4245.800087,692.969073,356.240158,933.328707,1236.603141
2022-01-01 02:00:00,177.989813,233.705153,67.769770,581.658785,720.831710,193.003198,177.283525,55.112962,243.787271,1327.045867,...,643.259691,368.590359,1326.841480,2011.894661,629.404499,4211.377977,687.350943,353.352001,925.761901,1226.577590
2022-01-01 03:00:00,179.144360,235.221103,68.209364,585.431769,725.507452,194.255131,178.433491,55.470458,245.368620,1335.653874,...,647.432255,370.981255,1335.448161,2024.945003,633.487189,4238.695472,691.809509,355.644051,931.766942,1234.533899
2022-01-01 04:00:00,167.330931,219.709770,63.711391,546.826274,677.664857,181.445243,166.666939,51.812534,229.188122,1247.576010,...,604.738222,346.517405,1247.383863,1891.412781,591.712745,3959.180510,646.189079,332.191592,870.322848,1153.124254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,264.689457,347.543992,100.780731,864.987416,1071.952102,287.015931,263.639134,81.958736,362.537155,1973.455923,...,956.594401,548.132394,1973.151978,2991.897668,935.990282,6262.759275,1022.162702,525.471362,1376.704719,1824.049092
2022-12-31 20:00:00,259.293224,340.458601,98.726111,847.352887,1050.098178,281.164527,258.264314,80.287840,355.146098,1933.223009,...,937.092278,536.957600,1932.925261,2930.901747,916.908216,6135.080185,1001.323836,514.758559,1348.637792,1786.862140
2022-12-31 21:00:00,235.836456,309.659268,89.794927,770.697740,955.101829,255.729188,234.900625,73.024661,323.018071,1758.335432,...,852.318924,488.382131,1758.064619,2665.759907,833.960798,5580.074733,910.739822,468.191310,1226.634280,1625.214990


 Let us import the heat demand profiles

In [5]:
heating_EDRP=pd.read_csv('REMA/domestic_EDRP/scaled with normalised profiles/hourly heat demand total_withGasboilers_MW.csv')
heating_EDRP_DHN=pd.read_csv('REMA/domestic_EDRP/DHN/scaled with normalised profiles/hourly heat demand total_withDHN_MW.csv')
heating_RHPP=pd.read_csv('REMA/domestic_RHPP/scaled with normalised profiles/hourly heat demand total_withHPs_MW.csv')

In [6]:
#My_network.loads_t.p_set.div(1e3).plot(figsize=(12,4),ylabel='GW')    # plotting the demand
#load_heat=My_network.loads_t.p_set.groupby(My_network.loads.carrier,axis=1).sum().div(1e3)
#My_network.loads_t.p_set.iloc[:,0:29].sum().div(1e3)

In [7]:
p0=My_network.buses.index[0:29].tolist()
p1=My_network.buses.index[29:58].tolist()
p0[1]

'Peterhead'

To approximate n-1 security and allow room for reactive power flows, don't allow any line to be loaded above 70% of thier thermal rating

In [8]:
contingency_factor = 0.7
My_network.lines.s_max_pu[0:99] *= contingency_factor

In [9]:
COPs=pd.read_csv('LOPF_data_heat/hourly COP_HPs.csv')
cop_1=COPs['Beauly COP'].values.tolist()

In [10]:
B0=My_network.buses.index[0:29].tolist()
B1=My_network.buses.index[29:58].tolist()


In [11]:
My_network.storage_units


,p_nom,carrier,marginal_cost,max_hours,efficiency_store,efficiency_dispatch,state_of_charge_initial,bus,control,type,...,capital_cost,build_year,lifetime,state_of_charge_initial_per_period,state_of_charge_set,cyclic_state_of_charge,cyclic_state_of_charge_per_period,standing_loss,inflow,p_nom_opt
StorageUnit,,,,,,,,,,,,,,,,,,,,,
Dinorwig,1431.724138,Pumped Storage Hydroelectric,0.0,5.808138e+00,0.865,0.865,8315.651565,Deeside,PQ,,...,0.0,0,inf,False,NaN,False,True,0.0,0.0,0.0
Ffestiniog,298.275862,Pumped Storage Hydroelectric,0.0,4.289375e+00,0.850,0.850,1279.417052,Deeside,PQ,,...,0.0,0,inf,False,NaN,False,True,0.0,0.0,0.0
Cruachan,440.000000,Pumped Storage Hydroelectric,0.0,1.779721e+01,0.865,0.865,7830.773160,Errochty,PQ,,...,0.0,0,inf,False,NaN,False,True,0.0,0.0,0.0
Foyers,300.000000,Pumped Storage Hydroelectric,0.0,2.316196e+01,0.865,0.865,6948.589110,Beauly,PQ,,...,0.0,0,inf,False,NaN,False,True,0.0,0.0,0.0
Beauly Battery,69.231494,Battery,0.0,1.253543e+00,0.950,0.950,86.784654,Beauly,PQ,,...,0.0,0,inf,False,NaN,False,True,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Stella West P2G,0.632079,P2G,500.0,1.000000e+09,0.950,0.950,0.000000,Stella West,PQ,,...,0.0,0,inf,False,NaN,False,True,0.0,0.0,0.0
Sundon/East Claydon P2G,0.203418,P2G,500.0,1.000000e+09,0.950,0.950,0.000000,Sundon/East Claydon,PQ,,...,0.0,0,inf,False,NaN,False,True,0.0,0.0,0.0
Th. Marsh/Stocksbridge P2G,0.070416,P2G,500.0,1.000000e+09,0.950,0.950,0.000000,Th. Marsh/Stocksbridge,PQ,,...,0.0,0,inf,False,NaN,False,True,0.0,0.0,0.0


In [12]:
My_network.consistency_check()    # will make to check if there is any inconsistency in the network that will lead to infeasibility

{'Norway'}


In [13]:
 My_network.lopf(My_network.snapshots,solver_name="gurobi", pyomo=False)
    #My_network.optimize(solver_name='gurobi')  # 

AC    29
DC     6
Name: carrier, dtype: int64
INFO:pypsa.linopf:Prepare linear problem
AC    29
DC     6
Name: carrier, dtype: int64
INFO:pypsa.linopf:Total preparation time: 129.8s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-29
Read LP format model from file C:\Users\salene\AppData\Local\Temp\pypsa-problem-n2px2e7q.lp
Reading time = 67.16 seconds
obj: 25298014 rows, 7866481 columns, 44516468 nonzeros
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 25298014 rows, 7866481 columns and 44516468 nonzeros
Model fingerprint: 0x9b9ec71e
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  Objective range  [1e+00, 1e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-06, 3e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 23248188 rows and 1657830 columns (presolve time = 6s) ...
Presolve removed 23250636 rows and 2780000 columns 


Crossover log...

 1048588 variables added to crossover basis                      685s

  188183 DPushes remaining with DInf 0.0000000e+00               687s
    5971 DPushes remaining with DInf 0.0000000e+00               691s
       0 DPushes remaining with DInf 0.0000000e+00               696s

  958333 PPushes remaining with PInf 6.7883238e-02               696s
  366395 PPushes remaining with PInf 2.2167293e-01               728s
  249745 PPushes remaining with PInf 2.0966451e-01               731s
  195846 PPushes remaining with PInf 1.6301097e-01               735s
  141408 PPushes remaining with PInf 1.6002666e-01               741s
  107200 PPushes remaining with PInf 1.0306978e-01               747s
   84614 PPushes remaining with PInf 5.8543525e-02               753s
   78428 PPushes remaining with PInf 5.4937930e-02               755s
   69290 PPushes remaining with PInf 3.8671718e-02               760s
   60451 PPushes remaining with PInf 3.4730823e-02               765s

INFO:pypsa.linopf:Optimization successful. Objective value: 5.60e+09


('ok', 'optimal')

In [14]:
My_network.generators_t.p

Generator,Drax - coal units,West Burton,Ratcliffe,Hartlepool,Heysham 1,Heysham 2,Hinkley Point B,Hunterston B,Torness,Sizewell B,...,Tidal lagoon S.W.Penisula,Tidal lagoon Deeside,Tidal lagoon Melksham,Tidal stream S.W.Penisula,Tidal stream Neilston,Tidal stream Peterhead,Tidal stream Deeside,Tidal stream Lovedean,Tidal stream Beauly,Wave power Beauly
snapshot,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0.0,0.0,0.0,738.998982,661.045503,773.298513,601.800859,0.0,501.144165,747.106144,...,0.000000,0.000000,0.000000,1.924931,0.0,0.0,1.924931,1.113965,0.0,0.0
2022-01-01 01:00:00,0.0,0.0,0.0,738.998982,661.045503,773.298513,601.800859,0.0,501.144165,747.106144,...,0.000000,0.000000,0.000000,5.000000,0.0,0.0,4.015884,4.015884,0.0,0.0
2022-01-01 02:00:00,0.0,0.0,0.0,738.998982,661.045503,773.298513,601.800859,0.0,501.144165,747.106144,...,0.000000,0.000000,0.395136,4.562799,0.0,0.0,3.056719,5.000000,0.0,0.0
2022-01-01 03:00:00,0.0,0.0,0.0,738.998982,661.045503,773.298513,601.800859,0.0,501.144165,747.106144,...,0.000000,0.000000,0.000000,1.351940,0.0,0.0,1.351940,4.015884,0.0,0.0
2022-01-01 04:00:00,0.0,0.0,0.0,738.998982,661.045503,773.298513,601.800859,0.0,501.144165,747.106144,...,0.000000,4.837500,0.000000,0.000000,0.0,0.0,0.000000,1.351940,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,0.0,0.0,0.0,738.998982,661.045503,773.298513,601.800859,0.0,501.144165,747.106144,...,3.417500,1.360667,6.902383,0.330064,0.0,0.0,0.000000,0.905694,0.0,0.0
2022-12-31 20:00:00,0.0,0.0,0.0,738.998982,661.045503,773.298513,601.800859,0.0,501.144165,747.106144,...,0.849583,0.000000,5.646998,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
2022-12-31 21:00:00,0.0,0.0,0.0,738.998982,661.045503,773.298513,601.800859,0.0,501.144165,747.106144,...,0.000000,0.000000,0.125616,1.351940,0.0,0.0,1.351940,0.439315,0.0,0.0


In [ ]:
My_network.generators.p_nom_opt.div(1e3) # GW  optimised capacity of generators in GW
My_network.storage_units.p_nom_opt.div(1e3)  # GW 
#My_network.links_t.efficiency 
My_network.storage_units

In [ ]:
#My_network.storage_units.p_nom_opt
#let us visualise this
#fig, ax=plt.subplots(1,1)
#fig.set_size_inches(19, 11)
#My_network.storage_units.p_nom_opt.div(1e3).plot.area()
#My_network.storage_units.p_nom_opt.div(1e3).plot()
#plt.legend()





In [ ]:
plt.figure(figsize=(19,11))
plt.bar(My_network.storage_units.index, My_network.storage_units.p_nom_opt.div(1e3).values)

plt.xticks(My_network.storage_units.index, rotation=90)
plt.ylabel('GW')
plt.grid(color='grey', linewidth=1, axis='both', alpha=0.5)
plt.title('Optimal Sizing of Thermal Energy Storage')
plt.show()

# Power output by generation type

In [ ]:
p_by_carrier = My_network.generators_t.p.groupby(
    My_network.generators.carrier, axis=1).sum()

storage_by_carrier = My_network.storage_units_t.p.groupby(
    My_network.storage_units.carrier, axis=1).sum()
# to show on graph set the negative storage values to zero
storage_by_carrier[storage_by_carrier < 0] = 0
p_by_carrier = pd.concat([p_by_carrier, storage_by_carrier], axis=1)



# interconnector exports
exports = My_network.loads_t.p
# multiply by negative one to convert it as a generator
# i.e. export is a positive load, but negative generator
exports['Interconnectors Export'] = exports.iloc[:, -6:].sum(axis=1) * -1
interconnector_export = exports[['Interconnectors Export']]

# group biomass stuff
p_by_carrier['Biomass'] = (
    p_by_carrier['Biomass (dedicated)'] + p_by_carrier['Biomass (co-firing)'] +
    p_by_carrier['Landfill Gas'] + p_by_carrier['Anaerobic Digestion'] +
    p_by_carrier['Sewage Sludge Digestion'])

# rename the hydro bit
p_by_carrier = p_by_carrier.rename(
    columns={'Large Hydro': 'Hydro'})
p_by_carrier = p_by_carrier.rename(
    columns={'Interconnector': 'Interconnectors Import'})

p_by_carrier

Graph the power output of the different generation types

In [ ]:
import hvplot.pandas

# Plotting Optimal Dispatch

In [ ]:
My_network.storage_units.p_nom_opt.div(1e3)*My_network.storage_units.max_hours      #GWh  storage full capcity
# Let us plot the optimal dispatch
p_by_carrier=My_network.generators_t.p.groupby(My_network.generators.carrier,axis=1).sum().div(1e3)
sto=My_network.storage_units_t.p.groupby(My_network.storage_units.carrier, axis=1).sum().div(1e3)
My_network.storage_units_t.p
carrier = pd.concat([p_by_carrier, sto], axis=1)
carrier

#fig, ax = plt.subplots(figsize=(6, 3))
#color = p_by_carrier.columns.map(My_network.carriers.color)
    
#p_by_carrier.where(p_by_carrier>0).loc[My_network.snapshots].plot.area(
      #  ax=ax,
        #linewidth=0,
        #color=color,
   # )
dispatch = pd.concat([My_network.generators_t.p, My_network.storage_units_t.p], axis=1).div(1e3)       #GW

dispatch.where(dispatch > 0, 0).hvplot.area(
    stacked=True, line_width=0,
    width=1300,
    height=350,
    hover=False,
    #color=[My_network.carriers.at[c, 'color'] for c in dispatch.columns],
    ylabel='electricity supply [GW]',
    ylim=(0,180)
)

In [ ]:
My_network.statistics.capex()
My_network.statistics.opex()
# total system cost
System_cost = My_network.statistics.capex() + My_network.statistics.opex(aggregate_time='sum')



# Alternative option for claculate the cost
#opex = My_network.snapshot_weightings.generators @ (My_network.generators.marginal_cost * My_network.generators_t.p).div(1e6) # M€/a
#capex = (My_network.generators.p_nom_opt * My_network.generators.capital_cost).div(1e6) # M€/a
#capex_storage = (My_network.storage_units.p_nom_opt * My_network.storage_units.capital_cost).div(1e6) # M€/a
#System_cost=pd.concat([opex, capex, capex_storage], axis=1).sum(axis=1)
#System_cost
#System_cost.plot.pie(figsize=(2,2))

#series_without_nan = System_cost.dropna()
#series_without_nan.plot.pie(figsize=(2,2))

# compute the cost per unit of electricity consumed:

In [ ]:
demand = My_network.snapshot_weightings.generators @ My_network.loads_t.p_set.sum(axis=1)
System_cost.sum()*1e6 / demand.sum()

In [ ]:
# Sensitivty analysis
#sensitivity = {}
#for co2 in [150,100,50,25,0]:
   # My_network.global_constraints.loc["CO2Limit", "constant"] = co2 * 1e6
   # My_network.optimize(solver_name='gurobi')
    

In [ ]:
df_network = pd.read_csv('LOPF_data_heat/buses.csv')
lon = df_network.iloc[28:]['x'].values
lat = df_network.iloc[28:]['y'].values
coordinates = np.zeros(shape=(len(lon), 2))
for i in range(len(lon)):
    coordinates[i][0] = lon[i]
    coordinates[i][1] = lat[i]

df_lines = pd.read_csv('LOPF_data_heat/lines.csv', index_col=0)
df_buses = pd.read_csv('LOPF_data_heat/buses.csv', index_col=0)

# Extract the coordinates of buses and lines for the sake of plotting
line_coordinates = []
for i in range(len(df_lines['bus0'].values)):

    bus0 = df_lines['bus0'].iloc[i]
    bus1 = df_lines['bus1'].iloc[i]
    bus0_coord = [df_buses.iloc[29:]['x'][bus0], df_buses.iloc[29:]['y'][bus0]]
    bus1_coord = [df_buses.iloc[29:]['x'][bus1], df_buses.iloc[29:]['y'][bus1]]
    line_coordinates.append([bus0_coord, bus1_coord])
    

In [ ]:
df_buses.iloc[28:].index
#df_lines['bus0'].iloc[98]

In [ ]:
# then using the above information, plot the network
fig = plt.figure()
fig.set_size_inches(15, 15)
ax = fig.add_subplot(1, 1, 1,
                     projection=ccrs.PlateCarree())

#extent = [-8.09782, 2.40511, 60, 49.5]
#ax.set_extent(extent)

ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.LAND)


for i in range(len(df_lines['bus0'].values)):
    ax.plot([line_coordinates[i][0][0], line_coordinates[i][1][0]],
            [line_coordinates[i][0][1], line_coordinates[i][1][1]],
            c='black')

# display the rconnection for the heat buses and also the electric bus Beally connection with the heat bus 3 in the figure
#for h in range(4):
 #   ax.plot([df_buses['x'].iloc[h],df_buses['x'].iloc[h+1]],
 #           [df_buses['y'].iloc[h],df_buses['y'].iloc[h+1]],
 #          color='red')
    
# ax.scatter(lon, lat, s=sizes * marker_scaler, c='black', edgecolors='black')

#ax.scatter(lon[0:4], lat[0:4], c='red', edgecolors='red')
#ax.scatter(lon[4:], lat[4:], c='black', edgecolors='black')
ax.scatter(lon, lat, c='black', edgecolors='black')
for i, txt in enumerate(df_buses.iloc[29:57].index):
    ax.annotate(txt, (lon[i], lat[i]),
                xytext=(0, 5),
                textcoords="offset points",
                ha='center',
                fontweight='extra bold',
                color='blue',
                fontsize='large')

ax.set_title('Reduced GB network with four heat buses  close to the first four electric buses', fontsize=20)
fig.tight_layout()

In [ ]:
My_network.links_t.p0

In [ ]:
My_network.links_t.efficiency

In [ ]:
My_network.links_t.p1

In [ ]:
p_by_carrier=My_network.generators_t.p.groupby(My_network.generators.carrier,axis=1).sum().div(1e3) #GW
p_by_carrier

In [ ]:
#My_network.generators.index[657]
#My_network.generators['p_nom_opt'][657]
#y_network.loads_t.p_set['load gas 1']
#My_network.generators_t.p

In [ ]:
fig, ax= plt.subplots(figsize=(19, 6))

p_by_carrier.plot(ax=ax,kind='area')
ax.legend(loc='upper center',bbox_to_anchor=(0.5,-0.05),
         fancybox=True,shadow=True,ncol=4)
#plt.legend(bbox_to_anchor=(1.0, 1.0))
ax.set_ylabel('GW')
ax.set_xlabel('')

In [ ]:
# interconnector exports
exports = My_network.loads_t.p
# multiply by negative one to convert it as a generator
# i.e. export is a positive load, but negative generator
exports['Interconnectors Export'] = exports.iloc[:, -6:].sum(axis=1) * -1

interconnector_export = exports[['Interconnectors Export']]


# group biomass stuff
p_by_carrier['Biomass'] = (
    p_by_carrier['Biomass (dedicated)'] + p_by_carrier['Biomass (co-firing)'] +
    p_by_carrier['Landfill Gas'] + p_by_carrier['Anaerobic Digestion'] +
    p_by_carrier['Sewage Sludge Digestion'])


# rename the hydro bit
p_by_carrier = p_by_carrier.rename(
    columns={'Large Hydro': 'Hydro'})
p_by_carrier = p_by_carrier.rename(
    columns={'Interconnector': 'Interconnectors Import'})

p_by_carrier

In [ ]:
# Add the heat pump dispatch to the generation data frame
p_hp=My_network.links_t.p1.sum(axis=1)
p_by_carrier['Heat Pump']=p_hp.values.tolist()

In [ ]:
p_by_carrier

In [ ]:
cols = ["Nuclear", 'Shoreline Wave', 'Biomass',
        'EfW Incineration',
        "Coal", "Oil", "Natural Gas", 'Hydro',
        "Wind Offshore", 'Wind Onshore', 'Solar Photovoltaics',
        'Interconnectors Import','Heat Pump'
       ]

In [ ]:
# we have aroudn 20 columns (generation type) but let us visualise the generation for the following types
#cols = ["Nuclear", 'Shoreline Wave', 'Biomass',
        #'EfW Incineration',
        #"Coal", "Oil", "Natural Gas", 'Pumped Storage Hydroelectric', 'Hydro',
        #"Wind Offshore", 'Wind Onshore', 'Solar Photovoltaics',
        #'Interconnectors Import','Heat Pump'
       # ]
p_by_carrier = p_by_carrier[cols]



#p_by_carrier.drop(
    #(p_by_carrier.max()[p_by_carrier.max() < 50.0]).index,
    #axis=1, inplace=True)

    

colors = {
        "Coal": "grey",
        "Diesel/Gas oil": "black",
        "Diesel/gas Diesel/Gas oil": "black",
        'Oil': 'black',
        'Anaerobic Digestion': 'green',
        'EfW Incineration': 'chocolate',
        'Sewage Sludge Digestion': 'green',
        'Landfill Gas': 'green',
        'Biomass (dedicated)': 'green',
        'Biomass (co-firing)': 'green',
        'Biomass': 'green',
        'Interconnectors Import': 'pink',
        "Sour gas": "lightcoral",
        "Natural Gas": "lightcoral",
        "Nuclear": "orange",
        'Shoreline Wave': 'aqua',
        'Tidal Barrage and Tidal Stream': 'aqua',
        'Hydro': "turquoise",
        "Large Hydro": "turquoise",
        "Small Hydro": "turquoise",
        "Pumped Storage Hydroelectric": "darkturquoise",
        "Wind Offshore": "lightskyblue",
        'Wind Onshore': 'deepskyblue',
        'Solar Photovoltaics': 'yellow',
        'Heat Pump': 'red'
}

    
fig,ax=plt.subplots(1,1)
fig.set_size_inches(15,8)
(p_by_carrier / 1e3).plot(
    kind="area", ax=ax, linewidth=0,
    color=[colors[col] for col in p_by_carrier.columns])   
    
    
    # Shrink current axis's height by 10% on the bottom
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.1,
                box.width, box.height * 0.9])

    
# Put a legend below current axis
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)

ax.set_ylabel("Dispatch in GW")
ax.set_xlabel("")

In [ ]:
My_network.storage_units_t.p['STES_Beauly']
My_network.storage_units_t.p_store['STES_Beauly'].plot()
My_network.storage_units_t.p_dispatch['STES_Beauly'].plot() 
My_network.generators_t.p['boiler 1']
My_network.links_t.p1['heat pump Beauly']
326.129451+130

In [ ]:
My_network.storage_units_t.state_of_charge

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)
p_storage=My_network.storage_units_t.p.sum(axis=1)
p_storage.plot(ax=ax,label='Thermal Energy Storage Dispatch',linewidth=1.5)

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)

p_storage = My_network.storage_units_t.p.sum(axis=1)
state_of_charge = My_network.storage_units_t.state_of_charge.sum(axis=1)
p_storage.plot(label="STES", ax=ax, linewidth=3)
state_of_charge.plot(label="State of charge", ax=ax, linewidth=3)
ax.legend()
ax.grid()
ax.set_ylabel("MWh")
ax.set_xlabel("")

In [ ]:
My_network.generators_t.p_max_pu

Plotting line loading
Look at the line loading stats and graph it

In [ ]:
My_network.snapshots[139]

In [ ]:
now = My_network.snapshots[139]

print("With the linear load flow, there is the following per unit loading:")
loading = My_network.lines_t.p0.loc[now] / My_network.lines.s_nom
loading.describe()

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})
fig.set_size_inches(15, 17)
My_network.plot(ax=ax, line_colors=abs(loading), line_cmap=plt.cm.jet, title="Line loading")

In [ ]:
fig,ax=plt.subplots(1,1)
fig.set_size_inches(15,8)
My_network.buses_t.marginal_price.iloc[:,0:29].plot(ax=ax)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=6)


# negative mariginal price? what does it tell? during summer?

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})
fig.set_size_inches(20, 10)

My_network.plot(ax=ax, line_widths=pd.Series(1, My_network.lines.index))
plt.hexbin(My_network.buses.x, My_network.buses.y,
           gridsize=20,
           C=My_network.buses_t.marginal_price.loc[now],
           cmap=plt.cm.jet)

# for some reason the colorbar only works with graphs plt.plot
# and must be attached plt.colorbar
cb = plt.colorbar()
cb.set_label('Locational Marginal Price (£/MWh)')

# Visualization

 plotting curtailment

In [ ]:
My_network.generators_t.p_max_pu.columns

In [ ]:
name='Wind Offshore Deeside'
type(My_network.generators_t.p_max_pu[name])
#My_network.generators.groupby("carrier").sum().at[carrier, "p_nom"]
#p_available = My_network.generators_t.p_max_pu
wind_offshore_deeside_available=My_network.generators.at[name,'p_nom']*My_network.generators_t.p_max_pu[name]
# next find the dispatched power at deeside
dispatched_wind_offshore_deeside=My_network.generators_t.p['Wind Offshore Deeside']
curtailed_wind_offshore_deeside=wind_offshore_deeside_available-dispatched_wind_offshore_deeside

capacity = My_network.generators.at[name, "p_nom"]
capacity_t=pd.Series(capacity,index=My_network.generators_t.p['Wind Offshore Deeside'].index,name='Wind Offshore Capacity')
capacity_t


wind_offshore_deeside_df=pd.concat([capacity_t,dispatched_wind_offshore_deeside.rename('Wind Offshore Deeside Dispatched'),wind_offshore_deeside_available.rename('Wind Offshore Deeside Available')],axis=1)


fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)
wind_offshore_deeside_df.plot(ax=ax)

ax.set_xlabel("")
ax.set_ylabel("Power [MW]")
ax.legend()

In [ ]:
carrier = "Wind Onshore"

capacity = My_network.generators.groupby("carrier").sum().at[carrier, "p_nom"]
p_available = My_network.generators_t.p_max_pu.multiply(My_network.generators["p_nom"])
p_available_by_carrier = p_available.groupby(My_network.generators.carrier, axis=1).sum()
p_curtailed_by_carrier = p_available_by_carrier - p_by_carrier
p_df = pd.DataFrame({carrier + " available": p_available_by_carrier[carrier],
                     carrier + " dispatched": p_by_carrier[carrier],
                     carrier + " curtailed": p_curtailed_by_carrier[carrier]})

p_df[carrier + " capacity"] = capacity
p_df["Wind Onshore curtailed"][p_df["Wind Onshore curtailed"] < 0.] = 0.
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)
p_df[[carrier + " dispatched", carrier + " curtailed"]].plot(kind="area", ax=ax, linewidth=0)

ax.set_xlabel("")
ax.set_ylabel("Power [MW]")
ax.legend()

Visualize the density of dwellings (number of dwellings) per each clustered local authority area. This will help the density of heat demand, connected to DHNs and HP in each clustered local authority area across GB.

In [ ]:
My_network.generators_t.marginal_cost
#My_network.objective
#opex = My_network.snapshot_weightings.generators @ (My_network.generators_t.p * My_network.generators.marginal_cost).div(1e6) # M€/a